In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
np.set_printoptions(threshold=np.inf)

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/gmbt18/CS176-Project/cedric-preproc/Exasens.csv").iloc[2:, [0, 6, 7, 8]]

# print(df.info())
# print(df.to_string())

df.dropna(inplace=True)
df = df.astype({"Diagnosis":"category","Gender":"int64", "Age":"int64", "Smoking":"int64"}).astype({"Gender":"category","Smoking":"category"})
df["Age"] = pd.cut(df["Age"], bins=[0, 40, 60, 100], include_lowest=True, labels=["Young", "Middle-aged", "Old"])
df = pd.get_dummies(df)

# print(df.info())
# print(df.to_string())

# from sklearn.utils import shuffle 
# df = shuffle(df)

print(df.info())
print(df.to_string())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399 entries, 2 to 400
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   Diagnosis_Asthma    399 non-null    uint8
 1   Diagnosis_COPD      399 non-null    uint8
 2   Diagnosis_HC        399 non-null    uint8
 3   Diagnosis_Infected  399 non-null    uint8
 4   Gender_0            399 non-null    uint8
 5   Gender_1            399 non-null    uint8
 6   Age_Young           399 non-null    uint8
 7   Age_Middle-aged     399 non-null    uint8
 8   Age_Old             399 non-null    uint8
 9   Smoking_1           399 non-null    uint8
 10  Smoking_2           399 non-null    uint8
 11  Smoking_3           399 non-null    uint8
dtypes: uint8(12)
memory usage: 4.8 KB
None
     Diagnosis_Asthma  Diagnosis_COPD  Diagnosis_HC  Diagnosis_Infected  Gender_0  Gender_1  Age_Young  Age_Middle-aged  Age_Old  Smoking_1  Smoking_2  Smoking_3
2                   0          

In [3]:
from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

print(frequent_itemsets.to_string())

      support                                                    itemsets  length
0    0.200501                                          (Diagnosis_Asthma)       1
1    0.197995                                            (Diagnosis_COPD)       1
2    0.401003                                              (Diagnosis_HC)       1
3    0.200501                                        (Diagnosis_Infected)       1
4    0.601504                                                  (Gender_0)       1
5    0.398496                                                  (Gender_1)       1
6    0.390977                                                 (Age_Young)       1
7    0.310777                                           (Age_Middle-aged)       1
8    0.298246                                                   (Age_Old)       1
9    0.448622                                                 (Smoking_1)       1
10   0.375940                                                 (Smoking_2)       1
11   0.175439   

In [6]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
rules["antecedents_length"] = rules["antecedents"].apply(lambda x: len(x))
rules["consequents_length"] = rules["consequents"].apply(lambda x: len(x))
rules.sort_values("lift",ascending=False)

rules

# print(len(rules))


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_length,consequents_length
0,(Age_Old),(Diagnosis_Asthma),0.298246,0.200501,0.090226,0.302521,1.508824,0.030427,1.146269,1,1
1,(Diagnosis_Asthma),(Age_Old),0.200501,0.298246,0.090226,0.450000,1.508824,0.030427,1.275917,1,1
2,(Diagnosis_COPD),(Gender_1),0.197995,0.398496,0.140351,0.708861,1.778839,0.061451,2.066035,1,1
3,(Gender_1),(Diagnosis_COPD),0.398496,0.197995,0.140351,0.352201,1.778839,0.061451,1.238047,1,1
4,(Age_Old),(Diagnosis_COPD),0.298246,0.197995,0.177945,0.596639,3.013403,0.118894,1.988304,1,1
...,...,...,...,...,...,...,...,...,...,...,...
483,"(Gender_1, Smoking_1)","(Age_Young, Diagnosis_Infected)",0.142857,0.125313,0.022556,0.157895,1.260000,0.004654,1.038690,2,2
484,"(Diagnosis_Infected, Smoking_1)","(Age_Young, Gender_1)",0.110276,0.117794,0.022556,0.204545,1.736460,0.009567,1.109058,2,2
485,(Age_Young),"(Gender_1, Diagnosis_Infected, Smoking_1)",0.390977,0.032581,0.022556,0.057692,1.770710,0.009818,1.026648,1,3
486,(Diagnosis_Infected),"(Age_Young, Gender_1, Smoking_1)",0.200501,0.080201,0.022556,0.112500,1.402734,0.006476,1.036394,1,3


In [7]:
from apyori import apriori as apy_apriori

df = df.replace(1, pd.Series(df.columns, df.columns))
ops = df.apply(lambda x: [v for v in x.values if v!=0], axis=1).values

association_rules = apy_apriori(ops, min_support=0.0045, min_confidence=0.2, min_lift=3, min_length=2)
association_rules = list(association_rules)

print(len(association_rules))
print(association_rules[0])

for item in association_rules:

    # first index of the inner list
    # Contains base item and add item
    # print(item)
    pair = item[0] 
    items = [x for x in pair]
    print("Rule: " + items[0] + " -> " + items[1])

    #second index of the inner list
    print("Support: " + str(item[1]))

    #third index of the list located at 0th
    #of the third index of the inner list

    print("Confidence: " + str(item[2][0][2]))
    print("Lift: " + str(item[2][0][3]))
    print("=====================================")

14
RelationRecord(items=frozenset({'Age_Old', 'Diagnosis_COPD'}), support=0.17794486215538846, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Age_Old'}), items_add=frozenset({'Diagnosis_COPD'}), confidence=0.5966386554621849, lift=3.013402829486225), OrderedStatistic(items_base=frozenset({'Diagnosis_COPD'}), items_add=frozenset({'Age_Old'}), confidence=0.8987341772151899, lift=3.013402829486225)])
Rule: Age_Old -> Diagnosis_COPD
Support: 0.17794486215538846
Confidence: 0.5966386554621849
Lift: 3.013402829486225
Rule: Age_Old -> Gender_1
Support: 0.13032581453634084
Confidence: 0.4369747899159664
Lift: 3.1134453781512605
Rule: Age_Old -> Diagnosis_COPD
Support: 0.015037593984962405
Confidence: 1.0
Lift: 3.3529411764705883
Rule: Age_Old -> Diagnosis_COPD
Support: 0.14285714285714285
Confidence: 0.4789915966386555
Lift: 3.0336134453781516
Rule: Age_Old -> Smoking_3
Support: 0.020050125313283207
Confidence: 0.8
Lift: 4.040506329113924
Rule: Diagnosis_COPD -> Gender_1
Support: 